# First-order methods
##  (Forward) Euler method

Assume that $y(t)$ has two continuous derivatives on $[a, b]$. Then, using Taylor’s Theorem, for $j = 0, 1, 2, \ldots ,N − 1$, we have,

\begin{align*}
y(t_{j+1}) = y(t_{j}) + hy′(t_{j}) + \frac{h^2}{2} y′′(\xi_{j}),
\end{align*}
for some $\xi_{j} \in(t_{j},t_{j+1})$ [2]. Now since $y(t)$ satisfies the differential equation (1),

\begin{align*}
y(t_{j+1}) = y(t_{j}) + hf(t_{j}, y(t_{j})) + \frac{h^2}{2} y′′(\xi_{j}).
\end{align*}

Therefore, Euler’s method approximates the solution of the ODE (1) at each grid point starting from $x_0$ where the solution is known as $y_0$. This process is carried forward iteratively until point $x_{n-1}$ is reached.

\begin{align*}
y_{j+1} = y(t_{j}) + hf(t_{j}, y_{j}).
\end{align*}

In [1]:
def ForwardEuler(f, y0, a, b, h= False, N=False):
    '''
    Inputs:
        f: the ODE y'=f(t,y(x))
        y0: the initial value
        x_range: interval [a,b]
        N: Number of points
        h: step size h
    '''
    if N:
        h = (b-a)/(N)
    if h:
        N = int((b-a)/h)
    t = np.linspace(a, b, N+1)
    y = np.zeros(t.shape, dtype=float)
    y[0] = y0
    for j in range(N):
        y[j+1] = y[j] + h*f(t[j], y[j])
    Table = pd.DataFrame({'t':t, 'y':y})
    return Table

<font color='Blue'><b>Example</b></font>: Conisder the initial value problem $\begin{cases}y'+2ty=te^{-t^2},\\ y(0) = 1 \end{cases},\quad 0 \leq x \leq 1$ with exact solution $y\left(x \right) = \left(1 + \dfrac{t^{2}}{2}\right) e^{- t^{2}}$. We use the forward Euler method for solving this IVP.

In [2]:
import numpy as np
import pandas as pd

# f(t, y(t)):
f = lambda t, y: t*np.exp(-t**(2))-2*t*y
(a, b) = (0, 1)
# the eact solution y(t)
y_exact = lambda t: (1+(t**2)/2)*np.exp(-t**2)
#
y0 = 1

In [5]:
Table = ForwardEuler(f = f, y0 = y0, a = a, b = b, N = 10)
Table['Exact'] = y_exact(Table['t'])
Table['Error'] =  np.abs(Table['Exact'] - Table['y'])
display(Table[1:].style.set_properties(subset=['Error'], **{'background-color': 'Lavender', 'color': 'Navy',
                                                'border-color': 'DarkGreen'}).format({'Error': "{:.4e}"}))

,t,y,Exact,Error
1,0.100000,1.000000,0.995000,4.9999e-03
2,0.200000,0.989900,0.980005,9.8953e-03
3,0.300000,0.969520,0.955058,1.4462e-02
4,0.400000,0.938767,0.920315,1.8452e-02
5,0.500000,0.897751,0.876151,2.1600e-02
6,0.600000,0.846916,0.823258,2.3658e-02
7,0.700000,0.787147,0.762720,2.4427e-02
8,0.800000,0.719830,0.696026,2.3804e-02
9,0.900000,0.646841,0.625026,2.1815e-02
10,1.000000,0.570447,0.551819,1.8627e-02


### Error Analysis

For meastureing error, we use the following measurement tool:
\begin{align*}
E_{h} = \max_{1 \leq j \leq \frac{b-a}{h}}\left| y(t_{j}) - y_{j}\right|
\end{align*}

In [7]:
h = [2**(-i) for i in range(3, 12)]
Cols = ['h', 'N', 'Eh']
Table = pd.DataFrame(np.zeros([len(h), len(Cols)], dtype = float), columns=Cols)
Table['h'] = h
Table['N'] = ((b-a)/Table['h']).astype(int)

for n in range(Table.shape[0]):
    TB = ForwardEuler(f = f, y0 = y0, a = a, b = b, h = Table['h'][n])
    Table.loc[n, 'Eh'] = np.max(np.abs(y_exact(TB['t'])[1:] - TB['y'][1:]))
    
display(Table.style.set_properties(subset=['h', 'N'], **{'background-color': 'PaleGreen', 'color': 'Black',
       'border-color': 'DarkGreen'}).format(dict(zip(Table.columns.tolist()[-1:], 3*["{:.4e}"]))))

,h,N,Eh
0,1.2500e-01,8.0000e+00,3.0904e-02
1,6.2500e-02,1.6000e+01,1.4889e-02
2,3.1250e-02,3.2000e+01,7.2969e-03
3,1.5625e-02,6.4000e+01,3.6122e-03
4,7.8125e-03,1.2800e+02,1.7972e-03
5,3.9062e-03,2.5600e+02,8.9636e-04
6,1.9531e-03,5.1200e+02,4.4762e-04
7,9.7656e-04,1.0240e+03,2.2367e-04
8,4.8828e-04,2.0480e+03,1.1180e-04


In [8]:
# This part is used for producing tables and figures
import sys
sys.path.insert(0,'..')
import hd_tools as hd
from bokeh.plotting import show

Loading BokehJS ...

In [14]:
hd.derivative_AccuracyOrder(vecs = [Table['Eh'].values],
                            labels = ['Forward Euler method'],
                            xlabel = r"$$i$$",
                            ylabel = r"$$\ln \left( E_{h_{i}} / E_{h_{i-1}}  \right)$$",
                            title = 'Order of accuracy: %s' % 'Forward Euler method',
                            legend_orientation = 'horizontal', ylim = [0.9, 1.1])

# Backward (Implicit) Euler method

Backward (Implicit) Euler method is defined as follows
\begin{align*}
y_{j+1} = y_{j} + hf(t_{j+1}, y_{j+1}).
\end{align*}
where $y_{j+1}$ for $j = 0, 1, 2, \ldots ,N − 1$ are approximated by solving the above equation in each step.

In [15]:
from sympy import symbols, solve
def BackwardEuler(f, y0, a, b, h= False, N=False):
    '''
    Inputs:
        f: the ODE y'=f(t,y(x))
        y0: the initial value
        x_range: interval [a,b]
        N: Number of points
        h: step size h
    '''
    if N:
        h = (b-a)/(N)
    if h:
        N = int((b-a)/h)
    t = np.linspace(a, b, N+1)
    y = np.zeros(t.shape, dtype=float)
    y[0] = y0
    b = symbols('b')
    for j in range(N):
        y[j+1] = solve(y[j] + h*f(t[j+1], b) - b)[0]
    Table = pd.DataFrame({'t':t, 'y':y})
    return Table

<font color='Blue'><b>Example</b></font>: Conisder the initial value problem $\begin{cases}y'+2ty=te^{-t^2},\\ y(0) = 1 \end{cases},\quad 0 \leq x \leq 1$ with exact solution $y\left(x \right) = \left(1 + \dfrac{t^{2}}{2}\right) e^{- t^{2}}$. We use the backward Euler method for solving this IVP.

In [16]:
# f(t, y(t)):
f = lambda t, y: t*np.exp(-t**(2))-2*t*y
(a, b) = (0, 1)
# the eact solution y(t)
y_exact = lambda t: (1+(t**2)/2)*np.exp(-t**2)
#
y0 = 1

In [17]:
Table = BackwardEuler(f = f, y0 = y0, a = a, b = b, N = 10)
Table['Exact'] = y_exact(Table['t'])
Table['Error'] =  np.abs(Table['Exact'] - Table['y'])
display(Table[1:].style.set_properties(subset=['Error'], **{'background-color': 'Lavender', 'color': 'Navy',
                                                'border-color': 'DarkGreen'}).format({'Error': "{:.4e}"}))

,t,y,Exact,Error
1,0.100000,0.990099,0.995000,4.9016e-03
2,0.200000,0.970495,0.980005,9.5107e-03
3,0.300000,0.941427,0.955058,1.3631e-02
4,0.400000,0.903252,0.920315,1.7063e-02
5,0.500000,0.856539,0.876151,1.9612e-02
6,0.600000,0.802142,0.823258,2.1116e-02
7,0.700000,0.741251,0.762720,2.1469e-02
8,0.800000,0.675374,0.696026,2.0652e-02
9,0.900000,0.606281,0.625026,1.8745e-02
10,1.000000,0.535891,0.551819,1.5928e-02


In [19]:
h = [2**(-i) for i in range(3, 8)]
Cols = ['h', 'N', 'Eh']
Table = pd.DataFrame(np.zeros([len(h), len(Cols)], dtype = float), columns=Cols)
Table['h'] = h
Table['N'] = ((b-a)/Table['h']).astype(int)

for n in range(Table.shape[0]):
    TB = BackwardEuler(f = f, y0 = y0, a = a, b = b, h = Table['h'][n])
    Table.loc[n, 'Eh'] = np.max(np.abs(y_exact(TB['t'])[1:] - TB['y'][1:]))
    
display(Table.style.set_properties(subset=['h', 'N'], **{'background-color': 'PaleGreen', 'color': 'Black',
       'border-color': 'DarkGreen'}).format(dict(zip(Table.columns.tolist()[-1:], 3*["{:.4e}"]))))

hd.derivative_AccuracyOrder(vecs = [Table['Eh'].values],
                            labels = ['Backward Euler method'],
                            xlabel = r"$$i$$",
                            ylabel = r"$$\ln \left( E_{h_{i}} / E_{h_{i-1}}  \right)$$",
                            title = 'Order of accuracy: %s' % ' Backward Euler method',
                            legend_orientation = 'horizontal', ylim = [0.9, 1.1])

,h,N,Eh
0,1.2500e-01,8.0000e+00,2.6255e-02
1,6.2500e-02,1.6000e+01,1.3750e-02
2,3.1250e-02,3.2000e+01,7.0121e-03
3,1.5625e-02,6.4000e+01,3.5410e-03


***
**References:**
1. Allaire, Grégoire, et al. Numerical linear algebra. Vol. 55. New York: Springer, 2008.
1. Burden, Richard L., and J. Douglas Faires. "Numerical analysis 8th ed." Thomson Brooks/Cole (2005).
1. Atkinson, Kendall E. An introduction to numerical analysis. John wiley & sons, 2008.
1. Khoury, Richard, and Douglas Wilhelm Harder. Numerical methods and modelling for engineering. Springer, 2016.
1. Zarowski, Christopher J. An introduction to numerical analysis for electrical and computer engineers. John Wiley & Sons, 2004.
1. [Euler method](https://en.wikipedia.org/wiki/Euler_method)
***